In [0]:
DRIVE = True

## Imports

In [2]:
# Basic imports ---
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf

# Keras / sklearn ---
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import AUC

# Misc ---
%load_ext autoreload
%autoreload 2
%matplotlib inline

! pip install ipython-autotime
%load_ext autotime

tf.enable_eager_execution()

if DRIVE:
    import os
    from google.colab import drive
    drive.mount('drive')
    os.chdir("drive/My Drive/SAM/7-projet")

# Self ---
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './scripts')

import msdi_text
import msdi_io

df = msdi_io.get_msdi_dataframe()

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=7b93198d40f5000358352e47dcfb5d24b7f12a9237f16b0b6468ce8617de3415
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


## Model

In [8]:
def create_model(vocab_size):
    num_filters = 32
    kernel_size = 3
    embedding_dim = 50
    maxlen = 100
    
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=vocab_size))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(15, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=[AUC(name='auc')])
    return model

time: 4.42 ms


# **Main**

In [5]:
vocab_size = 5000

X, y = msdi_text.load_bow(df, msdi_text.get_bow_dict("./data/lyrics/msx_lyrics_genre.txt", vocab_size))

time: 7.81 s


In [11]:
model = create_model(vocab_size)

model.fit(X["train"], y["train"], epochs=2)

Train on 10340 samples
Epoch 1/2
10340/10340 [==============================] - 14s 1ms/sample - loss: 2.3346 - auc: 0.7466
Epoch 2/2
10340/10340 [==============================] - 13s 1ms/sample - loss: 2.2236 - auc: 0.7830


time: 27 s


In [12]:
print(model.evaluate(X["test"], y["test"]))
print(model.evaluate(X["val"], y["val"]))
print(model.evaluate(X["train"], y["train"]))

2112/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================